# ***DCT & Inverted LSB based Steganography***

# **ZigZag Traversing**

In [ ]:

import numpy as np

# Zigzag scan of a matrix
# Argument is a two-dimensional matrix of any size,
# not strictly a square one.
# Function returns a 1-by-(m*n) array,
# where m and n are sizes of an input matrix,
# consisting of its items scanned by a zigzag method.

def zigzag(input):

    h = 0
    v = 0

    vmin = 0
    hmin = 0

    vmax = input.shape[0]
    hmax = input.shape[1]

    #print(vmax ,hmax )

    i = 0

    output = np.zeros(( vmax * hmax))

    while ((v < vmax) and (h < hmax)):

        if ((h + v) % 2) == 0:                 # going up

            if (v == vmin):
                #print(1)
                output[i] = input[v, h]        # if we got to the first line

                if (h == hmax):
                    v = v + 1
                else:
                    h = h + 1

                i = i + 1

            elif ((h == hmax -1 ) and (v < vmax)):   # if we got to the last column
                #print(2)
                output[i] = input[v, h]
                v = v + 1
                i = i + 1

            elif ((v > vmin) and (h < hmax -1 )):    # all other cases
                #print(3)
                output[i] = input[v, h]
                v = v - 1
                h = h + 1
                i = i + 1


        else:                                    # going down

            if ((v == vmax -1) and (h <= hmax -1)):       # if we got to the last line
                #print(4)
                output[i] = input[v, h]
                h = h + 1
                i = i + 1

            elif (h == hmin):                  # if we got to the first column
                #print(5)
                output[i] = input[v, h]

                if (v == vmax -1):
                    h = h + 1
                else:
                    v = v + 1

                i = i + 1

            elif ((v < vmax -1) and (h > hmin)):     # all other cases
                #print(6)
                output[i] = input[v, h]
                v = v + 1
                h = h - 1
                i = i + 1




        if ((v == vmax-1) and (h == hmax-1)):          # bottom right element
            #print(7)
            output[i] = input[v, h]
            break

    #print ('v:',v,', h:',h,', i:',i)
    return output




# Inverse zigzag scan of a matrix
# Arguments are: a 1-by-m*n array,
# where m & n are vertical & horizontal sizes of an output matrix.
# Function returns a two-dimensional matrix of defined sizes,
# consisting of input array items gathered by a zigzag method.


def inverse_zigzag(input, vmax, hmax):

    #print input.shape

    # initializing the variables
    h = 0
    v = 0

    vmin = 0
    hmin = 0

    output = np.zeros((vmax, hmax))

    i = 0

    while ((v < vmax) and (h < hmax)):
        #print ('v:',v,', h:',h,', i:',i)
        if ((h + v) % 2) == 0:                 # going up

            if (v == vmin):
                #print(1)

                output[v, h] = input[i]        # if we got to the first line

                if (h == hmax):
                    v = v + 1
                else:
                    h = h + 1

                i = i + 1

            elif ((h == hmax -1 ) and (v < vmax)):   # if we got to the last column
                #print(2)
                output[v, h] = input[i]
                v = v + 1
                i = i + 1

            elif ((v > vmin) and (h < hmax -1 )):    # all other cases
                #print(3)
                output[v, h] = input[i]
                v = v - 1
                h = h + 1
                i = i + 1


        else:                                    # going down

            if ((v == vmax -1) and (h <= hmax -1)):       # if we got to the last line
                #print(4)
                output[v, h] = input[i]
                h = h + 1
                i = i + 1

            elif (h == hmin):                  # if we got to the first column
                #print(5)
                output[v, h] = input[i]
                if (v == vmax -1):
                    h = h + 1
                else:
                    v = v + 1
                i = i + 1

            elif((v < vmax -1) and (h > hmin)):     # all other cases
                output[v, h] = input[i]
                v = v + 1
                h = h - 1
                i = i + 1




        if ((v == vmax-1) and (h == hmax-1)):          # bottom right element
            #print(7)
            output[v, h] = input[i]
            break


    return output


# **Image Processing**

In [ ]:
import cv2
import numpy as np

# Numpy Macros
HORIZ_AXIS = 1
VERT_AXIS  = 0

# Standard quantization table as defined by JPEG
JPEG_STD_LUM_QUANT_TABLE = np.asarray([
                                        [16, 11, 10, 16,  24, 40,   51,  61],
                                        [12, 12, 14, 19,  26, 58,   60,  55],
                                        [14, 13, 16, 24,  40, 57,   69,  56],
                                        [14, 17, 22, 29,  51, 87,   80,  62],
                                        [18, 22, 37, 56,  68, 109, 103,  77],
                                        [24, 36, 55, 64,  81, 104, 113,  92],
                                        [49, 64, 78, 87, 103, 121, 120, 101],
                                        [72, 92, 95, 98, 112, 100, 103,  99]
                                      ],
                                      dtype = np.float32)
# Image container class
class YCC_Image(object):
    def __init__(self, cover_image):
        self.height, self.width = cover_image.shape[:2]
        self.channels = [
                         split_image_into_8x8_blocks(cover_image[:,:,0]),
                         split_image_into_8x8_blocks(cover_image[:,:,1]),
                         split_image_into_8x8_blocks(cover_image[:,:,2]),
                        ]


def stitch_8x8_blocks_back_together(Nc, block_segments):
    '''
    Take the array of 8x8 pixel blocks and put them together by row so the numpy.block() method can sitch it back together
    '''
    image_rows = []
    temp = []
    for i in range(len(block_segments)):
        if i > 0 and not(i % int(Nc / 8)):
            image_rows.append(temp)
            temp = [block_segments[i]]
        else:
            temp.append(block_segments[i])
    image_rows.append(temp)

    return np.block(image_rows)


def split_image_into_8x8_blocks(image):
    blocks = []
    for vert_slice in np.vsplit(image, int(image.shape[0] / 8)):
        for horiz_slice in np.hsplit(vert_slice, int(image.shape[1] / 8)):
            blocks.append(horiz_slice)
    return blocks


# **File processing**

In [ ]:
import os

def get_file_paths(directory):
    file_paths = []
    
    # Traverse the directory and collect file paths
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
    return file_paths

def rename_file(file_path, new_directory):
    file_base, file_ext = os.path.splitext(file_path)
    file_direc, file_name = os.path.split(file_base)
    
    # Construct the new file name
    new_file_name = f"{new_directory}{file_name}{'_steg'}{file_ext}"
    return new_file_name

def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:  # Open in read mode with UTF-8 encoding
        file_content = file.read()  # Read the entire file
    return file_content

def stego_filename(file_path):
    file_base, file_ext = os.path.splitext(file_path)
    file_direc, file_name = os.path.split(file_base)
    return file_name

def cover_filename(file_path, new_directory):
    file_base, file_ext = os.path.splitext(file_path)
    file_direc, file_name = os.path.split(file_base)
    file_name = file_name.replace('_steg', '')
    new_file_name = f"{new_directory}{file_name}{file_ext}"
    return new_file_name


# **Security Encryption**

In [ ]:
import base64

def generate_key(password):
    
    # Generate a key based on the password
    password_padded = password.ljust(32, b'\0')[:32]  # Pad or trim to 32 bytes
    #print("Padded pass: ", password_padded,"\nLength: ", len(password_padded),"\n")
    key = base64.urlsafe_b64encode(password_padded)
    #print("Encrypted key: ", key,"\nKey Length: ", len(key))
    return key

def encrypt_message(message, key):
        return key+message

def decrypt_message(encrypted_message, key):
        #print("\nEntered key: ", key)
        #print("\n",encrypted_message)
        if key == encrypted_message[:len(key)]:
            return encrypted_message[len(key):]
        else:
            raise ValueError("Wrong decryption key entered!")


# **Data Embedding**

In [ ]:
import cv2
import struct
import bitstring
import numpy  as np
import time

#import zigzag as zz
#import image_processing as img

start_time = time.time()


def _embed_bits_into_dct(encoded_bits, dct_blocks):
    """ 
    Embeds encoded bits into quantized DCT coefficients.

    Args:
        encoded_bits (bitstring.BitArray): The bits to embed.
        dct_blocks (list): List of quantized DCT blocks.

    Returns:
        list: List of modified DCT blocks with embedded bits.
    """
    data_complete = False
    encoded_bits.pos = 0
    encoded_data_len = bitstring.pack('uint:32', len(encoded_bits))
    converted_blocks = []
    
    for current_dct_block in dct_blocks:
        for i in range(1, len(current_dct_block)):
            curr_coeff = np.int32(current_dct_block[i])
            if (curr_coeff > 1):
                curr_coeff = np.uint8(current_dct_block[i])
                
                if (encoded_bits.pos == (len(encoded_bits) - 1)): 
                    data_complete = True
                    break
                    
                    
                # Read bits separately to avoid BitStream subtraction
                if encoded_data_len.pos < len(encoded_data_len):
                    embed_bit = encoded_data_len.read(1).uint 
                else:
                    embed_bit = encoded_bits.read(1).uint

                # --- INVERTED LSB INSERTION ---
                embed_bit = 1 - embed_bit  # Invert the bit
                curr_coeff &= ~0x01         # Clear the LSB
                curr_coeff |= embed_bit     # Embed the inverted bit

                current_dct_block[i] = np.float32(curr_coeff)
        converted_blocks.append(current_dct_block)

    if not(data_complete): 
        raise ValueError("Data didn't fully embed into cover image!")

    return converted_blocks


def stego_execute(file_path, new_directory, key):
    start_time1 = time.time()
    print("\nProcessing ", file_path)
    try:
        NUM_CHANNELS = 3
        COVER_IMAGE_FILEPATH  = file_path 
        SECRET_MESSAGE_PATH = "/kaggle/input/secretmsg/secret_msg.txt"
        STEGO_IMAGE_FILEPATH  = rename_file(file_path, new_directory)
        SECRET_MESSAGE = read_text_file(SECRET_MESSAGE_PATH)

        gen_key = generate_key(key)


        raw_cover_image = cv2.imread(COVER_IMAGE_FILEPATH, flags=cv2.IMREAD_COLOR)
        height, width   = raw_cover_image.shape[:2]
        
        # Force Image Dimensions to be 8x8 compliant
        while(height % 8): height += 1 # Rows
        while(width  % 8): width  += 1 # Cols
        valid_dim = (width, height)
        padded_image    = cv2.resize(raw_cover_image, valid_dim)
        cover_image_f32 = np.float32(padded_image)
        cover_image_YCC = YCC_Image(cv2.cvtColor(cover_image_f32, cv2.COLOR_BGR2YCrCb))

        # Placeholder for holding stego image data
        stego_image = np.empty_like(cover_image_f32)

        for chan_index in range(NUM_CHANNELS):
            # FORWARD DCT STAGE
            dct_blocks = [cv2.dct(block) for block in cover_image_YCC.channels[chan_index]]

            # QUANTIZATION STAGE
            dct_quants = [np.around(np.divide(item, JPEG_STD_LUM_QUANT_TABLE)) for item in dct_blocks]

            # Sort DCT coefficients by frequency
            sorted_coefficients = [zigzag(block) for block in dct_quants]

            # Embed data in Luminance layer
            if (chan_index == 0):
                # DATA INSERTION STAGE
                secret_data = ""
                for char in SECRET_MESSAGE.encode('ascii'): secret_data += bitstring.pack('uint:8', char)
                secret_data = encrypt_message(secret_data, gen_key)
                embedded_dct_blocks   = _embed_bits_into_dct(secret_data, sorted_coefficients)
                desorted_coefficients = [inverse_zigzag(block, vmax=8,hmax=8) for block in embedded_dct_blocks]
            else:
                # Reorder coefficients to how they originally were
                desorted_coefficients = [inverse_zigzag(block, vmax=8,hmax=8) for block in sorted_coefficients]

            # DEQUANTIZATION STAGE
            dct_dequants = [np.multiply(data, JPEG_STD_LUM_QUANT_TABLE) for data in desorted_coefficients]

            # Inverse DCT Stage
            idct_blocks = [cv2.idct(block) for block in dct_dequants]

            # Rebuild full image channel
            stego_image[:,:,chan_index] = np.asarray(stitch_8x8_blocks_back_together(cover_image_YCC.width, idct_blocks))


        # Convert back to RGB (BGR) Colorspace
        stego_image_BGR = cv2.cvtColor(stego_image, cv2.COLOR_YCR_CB2BGR)

        # Clamp Pixel Values to [0 - 255]
        final_stego_image = np.uint8(np.clip(stego_image_BGR, 0, 255))

        # Write stego image
        cv2.imwrite(STEGO_IMAGE_FILEPATH, final_stego_image)
        print(file_path," embedding done.")
    
    except FileNotFoundError as e:
        print(f"Error: File not found - {e}")
    except cv2.error as e:
        print(f"Error: OpenCV error - {e}")
    except ValueError as e:
        print(f"Error: Data embedding failed - {e}")
    except Exception as e:  # Catch any other unexpected exceptions
        print(f"Error: An unexpected error occurred - {e}")
        
    end_time1 = time.time()

    # Calculate the elapsed time
    elapsed_time1 = end_time1 - start_time1

    # Print the time
    print(f"Time taken: {elapsed_time1:.2f} seconds\n")

directory = '/kaggle/input/test-dct'  # Input data directory
new_directory = '/kaggle/working/'    # Output data directory

# Get the list of file paths
file_paths = get_file_paths(directory)

key = input("Enter encryption key: ").encode()

# Print the file paths (optional)
for file_path in file_paths:
    stego_execute(file_path, new_directory, key)

print("Data embedding successful!")
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the result
print(f"Total time taken for execution: {elapsed_time:.2f} seconds")


# **Data Extraction**

In [ ]:
import cv2
import struct
import bitstring
import numpy  as np

#import zigzag as zz
#import image_processing as img
#import dct_embedding as src

start_time = time.time()

def _extract_bits_from_dct(dct_blocks):
    
    """
    Extracts encoded bits from quantized DCT coefficients.

    Args:
        dct_blocks (list): List of quantized DCT blocks.

    Returns:
        bitstring.BitArray: The extracted bits.
    """

    extracted_data = ""
    for current_dct_block in dct_blocks:
        for i in range(1, len(current_dct_block)):
            curr_coeff = np.int32(current_dct_block[i])
            if (curr_coeff > 1):
                extracted_bit = (np.uint8(curr_coeff) & 0x01) ^ 1
                extracted_data += bitstring.pack('uint:1', extracted_bit) 
    return extracted_data

def stego_extract(file_path, key):
    start_time1 = time.time()
    print("\nProcessing ", file_path)
    try:
        STEGO_IMAGE_FILEPATH  = file_path

        gen_key = generate_key(key)

        stego_image     = cv2.imread(STEGO_IMAGE_FILEPATH, flags=cv2.IMREAD_COLOR)
        stego_image_f32 = np.float32(stego_image)
        stego_image_YCC = YCC_Image(cv2.cvtColor(stego_image_f32, cv2.COLOR_BGR2YCrCb))

        # FORWARD DCT STAGE
        dct_blocks = [cv2.dct(block) for block in stego_image_YCC.channels[0]]  # Only care about Luminance layer

        # QUANTIZATION STAGE
        dct_quants = [np.around(np.divide(item, JPEG_STD_LUM_QUANT_TABLE)) for item in dct_blocks]

        # Sort DCT coefficients by frequency
        sorted_coefficients = [zigzag(block) for block in dct_quants]

        # DATA EXTRACTION STAGE
        recovered_data = _extract_bits_from_dct(sorted_coefficients)
        #print(recovered_data)

        # Check if enough bits are available to read the length of the secret message
        if len(recovered_data) < 32:
            raise ValueError("Not enough bits available to read the length of the secret message")

        recovered_data = bitstring.BitStream(recovered_data)
        data_len = int(recovered_data.read('uint:32') / 8)
        #print(data_len)

        # Extract secret message from DCT coefficients
        extracted_data = bytes()
        for _ in range(data_len): 
            if len(recovered_data) < 8:
                raise ValueError("Not enough bits available to read the secret message")
            extracted_data += struct.pack('>B', recovered_data.read('uint:8'))

        #print(extracted_data)
        decrypted_data = decrypt_message(extracted_data, gen_key)
        recovered_message = decrypted_data.decode('ascii')

        print("\nRecovered message from ",stego_filename(file_path)," : ", recovered_message)

    except FileNotFoundError as e:
        print(f"Error: File not found - {e}")
    except cv2.error as e:
        print(f"Error: OpenCV error - {e}")
    except ValueError as e:
        print(f"Error: Data extraction failed - {e}")
    except Exception as e:  # Catch any other unexpected exceptions
        print(f"Error: An unexpected error occurred - {e}")
    
    # Calculate the elapsed time
    end_time1 = time.time()
    elapsed_time1 = end_time1 - start_time1

    # Print the result
    print(f"\nTime taken: {elapsed_time1:.2f} seconds\n")


directory = '/kaggle/working/'  # Directory of stego images

# Get the list of file paths
file_paths = get_file_paths(directory)

key = input("Enter decryption key: ").encode()

# Run Extraction
for file_path in file_paths:
    stego_extract(file_path, key)
    
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the result
print(f"\n\nTotal time taken for execution: {elapsed_time:.2f} seconds")


# **PSNR, MSE & SSIM**

In [ ]:
import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, mean_squared_error, structural_similarity

def calculate_metrics(original_image, reconstructed_image):
    try:
        original_img = cv2.imread(original_image)
        reconstructed_img = cv2.imread(reconstructed_image)

        if original_img is None:
            raise FileNotFoundError(f"Could not load original image: {original_image}")
        if reconstructed_img is None:
            raise FileNotFoundError(f"Could not load reconstructed image: {reconstructed_image}")

        # Convert to grayscale (SSIM is typically calculated on grayscale)
        original_img_gray = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
        reconstructed_img_gray = cv2.cvtColor(reconstructed_img, cv2.COLOR_BGR2GRAY)

        # Ensure data types are suitable for the metrics
        original_img = original_img.astype(np.float32)
        reconstructed_img = reconstructed_img.astype(np.float32)
        original_img_gray = original_img_gray.astype(np.float32)
        reconstructed_img_gray = reconstructed_img_gray.astype(np.float32)

        if original_img.shape != reconstructed_img.shape:
            raise ValueError("Images must have the same dimensions.")

        mse = mean_squared_error(original_img, reconstructed_img)
        psnr = peak_signal_noise_ratio(original_img, reconstructed_img, data_range=255)
        ssim = structural_similarity(original_img_gray, reconstructed_img_gray, data_range=255, multichannel=False)

        #return {"psnr": psnr, "mse": mse, "ssim": ssim}

        print(f"PSNR: {psnr:.2f} dB")
        print(f"MSE: {mse:.4f}")
        print(f"SSIM: {ssim:.4f}")


    except FileNotFoundError as e:
        print(f"Error: File not found - {e}")
    except cv2.error as e:
        print(f"Error: OpenCV error - {e}")
    except ValueError as e:
        print(f"Error: Data embedding failed - {e}")
    except Exception as e:  # Catch any other unexpected exceptions
        print(f"Error: An unexpected error occurred - {e}")

        
stego_directory = "/kaggle/working/"
cover_directory = "/kaggle/input/test-dct/"

file_paths = get_file_paths(stego_directory)

for file_path in file_paths:
    original_image_path = cover_filename(file_path, cover_directory)
    reconstructed_image_path = file_path
    print("\nProcessing",original_image_path,"with its stego\n")
    calculate_metrics(original_image_path, reconstructed_image_path)

# **Comparison Image Plotting**

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def plot_images_side_by_side(image1_path, image2_path):
    # Load images
    img1 = cv2.imread(image1_path)
    img2 = cv2.imread(image2_path)
    
    # Ensure both images have the same dimensions
    if img1.shape != img2.shape:
        img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))
    
    # Convert images from BGR to RGB
    img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    
    # Convert images from BGR to YCbCr
    img1_ycbcr = cv2.cvtColor(img1, cv2.COLOR_BGR2YCrCb)
    img2_ycbcr = cv2.cvtColor(img2, cv2.COLOR_BGR2YCrCb)
    
    # Compute the difference image
    diff_image = cv2.absdiff(img1, img2)
    diff_image_rgb = cv2.cvtColor(diff_image, cv2.COLOR_BGR2RGB)
    
    # Highlight significant differences
    _, diff_highlight = cv2.threshold(cv2.cvtColor(diff_image, cv2.COLOR_BGR2GRAY), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    diff_highlight_colored = cv2.cvtColor(diff_highlight, cv2.COLOR_GRAY2RGB)
    
    # Highlight bit manipulations
    bit_diff = cv2.bitwise_xor(img1, img2)
    bit_diff_highlight = cv2.cvtColor(bit_diff, cv2.COLOR_BGR2RGB)
    
    # Plot images side by side
    fig, axes = plt.subplots(3, 2, figsize=(12, 24))
    
        # Original RGB Images
    axes[0, 0].imshow(img1_rgb)
    axes[0, 0].set_title('Cover - RGB')
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(img2_rgb)
    axes[0, 1].set_title('Stego - RGB')
    axes[0, 1].axis('off')
    
    # YCbCr Images
    axes[1, 0].imshow(img1_ycbcr)
    axes[1, 0].set_title('Cover - YCbCr')
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(img2_ycbcr)
    axes[1, 1].set_title('Stego - YCbCr')
    axes[1, 1].axis('off')
    
    # Highlighted Differences
    axes[2, 0].imshow(diff_highlight_colored)
    axes[2, 0].set_title('Highlighted Differences')
    axes[2, 0].axis('off')
    
    # Bit Manipulation Highlight
    axes[2, 1].imshow(bit_diff_highlight)
    axes[2, 1].set_title('Bit Manipulation Highlight')
    axes[2, 1].axis('off')

    plt.tight_layout()
    plt.show()

stego_directory = "/kaggle/working/"
cover_directory = "/kaggle/input/test-dct/"

file_paths = get_file_paths(stego_directory)

for file_path in file_paths:
    image1_path = cover_filename(file_path, cover_directory)
    image2_path = file_path
    plot_images_side_by_side(image1_path, image2_path)
